# W14 — LLM i AI w analizie danych
## Notebook demonstracyjny — działa BEZ kluczy API

**Programowanie w Pythonie II** | Wykład 14  
**Kierunek:** Analityka danych w biznesie  
**Politechnika Opolska**

---

> **UWAGA:** Ten notebook używa symulowanych odpowiedzi (mock responses) zamiast
> prawdziwych wywołań API. Pokazuje **dokładnie takie wyniki**, jakie dostałby kod
> z prawdziwym kluczem API. Do uruchomienia z prawdziwym API potrzeba zmiennej
> środowiskowej `OPENAI_API_KEY` lub `ANTHROPIC_API_KEY`.

### Zawartość
1. Tokenizacja — jak LLM widzi tekst
2. Temperatura — deterministyczny vs kreatywny
3. Porównanie modeli GPT / Claude / Gemini
4. Struktura wywołania API (OpenAI + Anthropic)
5. Generowanie kodu analitycznego przez AI
6. Interpretacja wyników statystycznych przez AI
7. Czyszczenie danych z pomocą AI
8. Automatyczne podsumowania (executive summary)

In [ ]:
# === SETUP ===
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import json
import textwrap

np.random.seed(42)

# Styl wykresów
plt.rcParams['figure.figsize'] = (11, 5)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

print("numpy:", np.__version__)
print("pandas:", pd.__version__)
print("Środowisko gotowe — notebook działa bez kluczy API.")

---
## Sekcja 1: Tokenizacja — jak LLM widzi tekst

LLM nie czyta liter ani słów — czyta **tokeny**. Token to fragment tekstu,
zwykle 3-4 znaki w angielskim, więcej w polskim (fleksja tworzy więcej wariantów).

**Dlaczego to ważne dla analityka:**
- API jest rozliczane **per token** (input + output osobno)
- Modele mają limit kontekstu wyrażony w tokenach (~100k-200k tokenów)
- Rozumienie tokenów pomaga optymalizować koszty

In [ ]:
# === SEKCJA 1: TOKENIZACJA (symulowana) ===

def symuluj_tokenizacje(tekst, srednia_znaki_na_token=3.5):
    """Przybliżona symulacja tokenizacji — rzeczywista zależy od modelu."""
    # Prosta heurystyka: podział na słowa + znaki interpunkcyjne jako osobne tokeny
    import re
    czesci = re.findall(r"[\w']+|[^\s\w']", tekst)
    # Dłuższe słowa rozbijamy na fragmenty (przybliżenie)
    tokeny = []
    for czesc in czesci:
        if len(czesc) <= 4:
            tokeny.append(czesc)
        elif len(czesc) <= 8:
            tokeny.append(czesc[:4])
            tokeny.append(czesc[4:])
        else:
            # Trójki
            for i in range(0, len(czesc), 4):
                tokeny.append(czesc[i:i+4])
    return tokeny


# Przykłady tekstów analitycznych
przyklady = [
    ("EN", "Data analysis with Python"),
    ("PL", "Analiza danych w Pythonie"),
    ("EN", "SELECT COUNT(*) FROM orders WHERE status='cancelled'"),
    ("PL", "Proszę przeprowadź klasteryzację klientów metodą k-średnich"),
    ("PL", "Programowanie"),
    ("EN", "Programming"),
]

print(f"{'Język':4} | {'Tekst (skrócony)':52} | {'~Tokeny':8}")
print("-" * 72)
for lang, tekst in przyklady:
    tokeny = symuluj_tokenizacje(tekst)
    skrocony = tekst[:50] + "..." if len(tekst) > 50 else tekst
    print(f"{lang:4} | {skrocony:52} | {len(tokeny):8}")

print()
print("Uwaga: To jest przybliżona symulacja. Rzeczywista tokenizacja")
print("zależy od modelu (BPE, WordPiece, SentencePiece).")

In [ ]:
# === WYKRES: Gęstość tokenizacji PL vs EN ===

pary = [
    ("programowanie",         "programming"),
    ("analiza danych",        "data analysis"),
    ("przetwarzanie",         "processing"),
    ("klasteryzacja",         "clustering"),
    ("przeprowadź analizę",   "run analysis"),
    ("uczenie maszynowe",     "machine learning"),
]

etykiety = [pl for pl, en in pary]
tokeny_pl = [len(symuluj_tokenizacje(pl)) for pl, en in pary]
tokeny_en = [len(symuluj_tokenizacje(en)) for pl, en in pary]

x = np.arange(len(etykiety))
szerokosc = 0.35

fig, ax = plt.subplots(figsize=(11, 5))
belki_pl = ax.bar(x - szerokosc/2, tokeny_pl, szerokosc, label='Polski', color='#e74c3c', alpha=0.8)
belki_en = ax.bar(x + szerokosc/2, tokeny_en, szerokosc, label='Angielski', color='#3498db', alpha=0.8)

ax.set_xticks(x)
ax.set_xticklabels(etykiety, rotation=15, ha='right', fontsize=10)
ax.set_ylabel('Przybliżona liczba tokenów')
ax.set_title('Tokenizacja: Polski vs Angielski\n(języki fleksyjne = więcej tokenów = wyższy koszt API)',
             fontsize=11)
ax.legend()
ax.set_ylim(0, max(max(tokeny_pl), max(tokeny_en)) + 2)

for belka in belki_pl:
    ax.text(belka.get_x() + belka.get_width()/2., belka.get_height() + 0.1,
            f'{int(belka.get_height())}', ha='center', va='bottom', fontsize=9)
for belka in belki_en:
    ax.text(belka.get_x() + belka.get_width()/2., belka.get_height() + 0.1,
            f'{int(belka.get_height())}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()
plt.close()

---
## Sekcja 2: Temperatura — deterministyczny vs kreatywny

Temperatura kontroluje "losowość" przy wyborze następnego tokenu.

| Temperatura | Zastosowanie | Przykład |
|-------------|-------------|----------|
| 0.0 | Kod, SQL, obliczenia | Generowanie funkcji Python |
| 0.3–0.7 | Wyjaśnienia, raporty | Interpretacja wyników |
| 1.0–1.5 | Kreatywność | Brainstorming, nazwy kampanii |

In [ ]:
# === SEKCJA 2: SYMULACJA EFEKTU TEMPERATURY ===

# Symulujemy: jak różna temperatura wpływa na odpowiedzi
# (prawdziwe modele są bardziej złożone, ale koncepcja jest oddana)

pytanie = "Jak nazywa się metoda analizy skupień w uczeniu maszynowym?"

odpowiedzi = {
    0.0: [
        "Klasteryzacja (ang. clustering).",
        "Klasteryzacja (ang. clustering).",
        "Klasteryzacja (ang. clustering).",
    ],
    0.7: [
        "Klasteryzacja (clustering). Popularne algorytmy to K-Means i DBSCAN.",
        "Analiza skupień (clustering) — grupowanie podobnych obserwacji. Np. K-Means.",
        "Klasteryzacja. W Pythonie: sklearn.cluster.KMeans lub AgglomerativeClustering.",
    ],
    1.5: [
        "To magia grupowania danych — klasteryzacja! Niczym sortowanie klocków Lego.",
        "Mówię o klasteryzacji — ale możesz to też nazwać 'sztuką znajdowania rodzin w danych'.",
        "Analiza skupień, klasteryzacja, segment analysis — wiele nazw dla jednej idei!",
    ],
}

print(f"Pytanie: {pytanie}")
print("=" * 65)

for temp, odp_lista in odpowiedzi.items():
    opis = {0.0: "KOD/FAKTY", 0.7: "ZRÓWNOWAŻONA", 1.5: "KREATYWNA"}
    print(f"\nTemperatura = {temp} ({opis[temp]}):")
    for i, odp in enumerate(odp_lista, 1):
        print(f"  Próba {i}: {odp}")

print()
print("WNIOSEK: Przy temp=0.0 każda próba daje IDENTYCZNĄ odpowiedź.")
print("         Przy temp=1.5 każda próba jest inna — kreatywna, ale mniej przewidywalna.")
print("         W analizie danych: ZAWSZE temperatura blisko 0 dla kodu i obliczeń.")

---
## Sekcja 3: Porównanie modeli — GPT vs Claude vs Gemini

In [ ]:
# === SEKCJA 3: PORÓWNANIE MODELI ===

# Dane porównawcze (orientacyjne, stan na 2025)
modele = [
    {
        "nazwa": "GPT-4o",
        "firma": "OpenAI",
        "kontekst_k_tokenow": 128,
        "mocne_strony": ["Kod i matematyka", "Integracja Azure/MS", "Function calling"],
        "slabsze_strony": ["Droższy", "Halucynacje przy faktach"],
        "cena_input_per_1m": 2.50,
        "cena_output_per_1m": 10.00,
        "kolor": "#10a37f",
    },
    {
        "nazwa": "Claude Sonnet",
        "firma": "Anthropic",
        "kontekst_k_tokenow": 200,
        "mocne_strony": ["Długie dokumenty", "Śledzenie instrukcji", "Mniej halucynacji"],
        "slabsze_strony": ["Mniej integracji", "Bywa ostrożny"],
        "cena_input_per_1m": 3.00,
        "cena_output_per_1m": 15.00,
        "kolor": "#cc785c",
    },
    {
        "nazwa": "Gemini 1.5",
        "firma": "Google",
        "kontekst_k_tokenow": 1000,
        "mocne_strony": ["Google Workspace", "Multimodalność", "BigQuery integracja"],
        "slabsze_strony": ["Mniej dojrzały API", "Mniejsza społeczność"],
        "cena_input_per_1m": 1.25,
        "cena_output_per_1m": 5.00,
        "kolor": "#4285f4",
    },
]

print(f"{'Model':15} {'Firma':12} {'Kontekst':12} {'Input $/1M':12} {'Output $/1M':12}")
print("-" * 65)
for m in modele:
    print(f"{m['nazwa']:15} {m['firma']:12} {m['kontekst_k_tokenow']:>6} k tok   "
          f"${m['cena_input_per_1m']:>6.2f}       ${m['cena_output_per_1m']:>6.2f}")

print()
print("Mocne strony:")
for m in modele:
    print(f"  {m['nazwa']:15}: {', '.join(m['mocne_strony'])}")

In [ ]:
# === WYKRES: Kontekst i koszty modeli ===

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

nazwy = [m['nazwa'] for m in modele]
kolory = [m['kolor'] for m in modele]
konteksty = [m['kontekst_k_tokenow'] for m in modele]
ceny_input = [m['cena_input_per_1m'] for m in modele]
ceny_output = [m['cena_output_per_1m'] for m in modele]

# Panel 1: Kontekst
bars1 = axes[0].barh(nazwy, konteksty, color=kolory, alpha=0.85, edgecolor='white')
axes[0].set_xlabel('Okno kontekstu (tysiące tokenów)')
axes[0].set_title('Okno kontekstu\n(im większe, tym dłuższy dokument model widzi naraz)')
for bar, val in zip(bars1, konteksty):
    axes[0].text(val + 10, bar.get_y() + bar.get_height()/2,
                 f'{val}k', va='center', fontsize=10)
axes[0].set_xlim(0, max(konteksty) * 1.2)

# Panel 2: Koszty
x = np.arange(len(nazwy))
szer = 0.35
axes[1].bar(x - szer/2, ceny_input, szer, label='Input ($/1M tok)', color=kolory, alpha=0.6)
axes[1].bar(x + szer/2, ceny_output, szer, label='Output ($/1M tok)', color=kolory, alpha=0.9)
axes[1].set_xticks(x)
axes[1].set_xticklabels(nazwy)
axes[1].set_ylabel('Cena (USD / milion tokenów)')
axes[1].set_title('Porównanie kosztów API\n(orientacyjne ceny, stan 2025)')
axes[1].legend()

plt.tight_layout()
plt.show()
plt.close()

---
## Sekcja 4: Struktura wywołania API

Kod poniżej pokazuje **jak wygląda wywołanie API** — komentarze wskazują co zmienić
żeby użyć prawdziwego klucza zamiast mock odpowiedzi.

In [ ]:
# === SEKCJA 4A: MOCK API — OpenAI struktura ===

# ---------------------------------------------------------------
# WERSJA PRODUKCYJNA (wymaga klucza API):
#
# from openai import OpenAI
# client = OpenAI()  # czyta OPENAI_API_KEY z zmiennych środowiskowych
#
# response = client.chat.completions.create(
#     model="gpt-4o",
#     messages=[
#         {"role": "system", "content": system_msg},
#         {"role": "user",   "content": user_msg}
#     ],
#     temperature=0.1,
#     max_tokens=500
# )
# odpowiedz = response.choices[0].message.content
# ---------------------------------------------------------------

# WERSJA MOCK (działa bez klucza — do celów dydaktycznych):

class MockOpenAIResponse:
    """Symuluje strukturę odpowiedzi OpenAI API."""
    def __init__(self, content, model="gpt-4o", input_tokens=150, output_tokens=80):
        self.choices = [type('Choice', (), {
            'message': type('Message', (), {'content': content, 'role': 'assistant'})()
        })()]
        self.model = model
        self.usage = type('Usage', (), {
            'prompt_tokens': input_tokens,
            'completion_tokens': output_tokens,
            'total_tokens': input_tokens + output_tokens
        })()

def mock_openai_call(system_msg, user_msg, model="gpt-4o",
                      temperature=0.1, max_tokens=500):
    """Mock wywołania OpenAI — zwraca strukturę identyczną z prawdziwym API."""
    # W produkcji: zamień na prawdziwe wywołanie (patrz komentarz powyżej)
    from_tokens = len(system_msg.split()) + len(user_msg.split())

    # Predefiniowane mock odpowiedzi
    mock_answers = {
        "p-wartosc": """P-wartość (p-value) to prawdopodobieństwo uzyskania
wyniku co najmniej tak ekstremalnego jak zaobserwowany, przy założeniu
że hipoteza zerowa jest prawdziwa.

Prościej: jeśli p < 0.05, różnica którą widzimy jest zbyt duża
żeby być przypadkiem — odrzucamy hipotezę zerową.""",
        "default": "Odpowiedź modelu AI na zadane pytanie analityczne."
    }

    klucz = "p-wartosc" if "p-wartość" in user_msg.lower() or "p-value" in user_msg.lower() else "default"
    tresc = mock_answers[klucz]

    return MockOpenAIResponse(
        content=tresc,
        model=model,
        input_tokens=from_tokens,
        output_tokens=len(tresc.split())
    )


# === Przykład użycia ===
system_msg = "Jesteś asystentem analityka danych. Odpowiadaj po polsku, zwięźle."
user_msg   = "Wyjaśnij czym jest p-wartość w jednym akapicie."

response = mock_openai_call(system_msg, user_msg)

print("=== Wywołanie API (mock) ===")
print(f"Model:       {response.model}")
print(f"Tokeny:      {response.usage.prompt_tokens} input + {response.usage.completion_tokens} output")
print(f"Koszt (~):   ${response.usage.total_tokens / 1_000_000 * 2.50:.6f} USD")
print()
print("Odpowiedź modelu:")
print("-" * 50)
print(response.choices[0].message.content)

In [ ]:
# === SEKCJA 4B: MOCK API — Anthropic struktura ===

# ---------------------------------------------------------------
# WERSJA PRODUKCYJNA (wymaga klucza API):
#
# import anthropic
# client = anthropic.Anthropic()  # czyta ANTHROPIC_API_KEY
#
# message = client.messages.create(
#     model="claude-sonnet-4-5",
#     max_tokens=500,
#     system=system_msg,
#     messages=[{"role": "user", "content": user_msg}]
# )
# odpowiedz = message.content[0].text
# ---------------------------------------------------------------

class MockAnthropicResponse:
    """Symuluje strukturę odpowiedzi Anthropic API."""
    def __init__(self, content, model="claude-sonnet-4-5"):
        self.content = [type('Block', (), {'text': content, 'type': 'text'})()]
        self.model = model
        self.stop_reason = "end_turn"


def mock_anthropic_call(system_msg, user_msg, model="claude-sonnet-4-5", max_tokens=500):
    """Mock wywołania Anthropic — identyczna struktura z prawdziwym API."""
    tresc = """Korelacja r=0.15 jest **słaba** (prawie brak związku liniowego).

Według skali Cohena:
- r < 0.1 → zaniedbywalna
- 0.1–0.3 → słaba
- 0.3–0.5 → umiarkowana
- r > 0.5 → silna

r=0.15 mieści się w przedziale słabej korelacji.
Oznacza to, że wiek wyjaśnia tylko r² = 2.25% wariancji satysfakcji — 
inne czynniki mają o wiele większe znaczenie."""
    return MockAnthropicResponse(content=tresc, model=model)


# Demonstracja
system = "Jesteś ekspertem statystyki. Odpowiadaj zwięźle, po polsku."
user   = "Czy korelacja r=0.15 między wiekiem a satysfakcją z pracy jest silna?"

msg = mock_anthropic_call(system, user)

print("=== Wywołanie Anthropic API (mock) ===")
print(f"Model: {msg.model} | Stop reason: {msg.stop_reason}")
print()
print("Odpowiedź Claude:")
print("-" * 50)
print(msg.content[0].text)

In [ ]:
# === SEKCJA 4C: STRUCTURED OUTPUT ===

def mock_structured_call(pytanie, dane_kontekst=""):
    """
    Symuluje wywołanie z prośbą o odpowiedź w JSON.
    W produkcji: użyj response_format={"type": "json_object"} w OpenAI
    lub odpowiednio sformułowanego system message.
    """
    # Mock JSON responses per typ pytania
    if "korelacja" in pytanie.lower() and "0.15" in pytanie:
        return {
            "wynik": "Korelacja r=0.15 jest bardzo słaba — wiek praktycznie nie wyjaśnia satysfakcji z pracy.",
            "pewnosc": 95,
            "ostrzezenie": "Korelacja liniowa nie wyklucza związku nieliniowego (U-kształt) — warto sprawdzić scatterplot."
        }
    elif "p-warto" in pytanie.lower() and ("0.03" in pytanie or "0.031" in pytanie):
        return {
            "wynik": "p=0.031 < 0.05 — wynik statystycznie istotny, odrzucamy hipotezę zerową.",
            "pewnosc": 99,
            "ostrzezenie": None
        }
    else:
        return {
            "wynik": "Odpowiedź na zadane pytanie analityczne.",
            "pewnosc": 80,
            "ostrzezenie": "Pytanie zbyt ogólne — podaj więcej kontekstu."
        }


# Test 1: pytanie o korelację
odpowiedz_json = mock_structured_call(
    "Czy korelacja r=0.15 między wiekiem a satysfakcją z pracy jest silna?"
)

print("=== Structured Output (JSON) ===")
print(json.dumps(odpowiedz_json, ensure_ascii=False, indent=2))
print()

# Programowe użycie wyników
print(f"Wynik:      {odpowiedz_json['wynik']}")
print(f"Pewność:    {odpowiedz_json['pewnosc']}%")
if odpowiedz_json.get('ostrzezenie'):
    print(f"Ostrzeżenie: {odpowiedz_json['ostrzezenie']}")

print()
print("Zaleta structured output: JSON gotowy do dalszego przetwarzania")
print("— nie trzeba parsować tekstu, pola są przewidywalne.")

---
## Sekcja 5: Generowanie kodu analitycznego przez AI

Technika 1: Precyzyjny prompt z kontekstem → kod gotowy do użycia

In [ ]:
# === SEKCJA 5: GENEROWANIE KODU — DATASET I DEMONSTRACJA ===

# Tworzymy dataset który będzie używany w kolejnych sekcjach
np.random.seed(42)
n = 500

df_sprzedaz = pd.DataFrame({
    'data_zamowienia': pd.date_range('2024-01-01', periods=n, freq='D')[:n],
    'klient_id': np.random.randint(1, 101, n),
    'produkt_kategoria': np.random.choice(
        ['elektronika', 'odzież', 'dom_i_ogrod', 'sport', 'ksiazki'],
        n, p=[0.30, 0.25, 0.20, 0.15, 0.10]
    ),
    'wartosc_zamowienia': np.round(np.random.lognormal(5.5, 0.8, n), 2),
    'status': np.random.choice(
        ['zrealizowane', 'anulowane', 'zwrócone'],
        n, p=[0.78, 0.12, 0.10]
    ),
})
df_sprzedaz['data_zamowienia'] = pd.to_datetime(df_sprzedaz['data_zamowienia'])

print("Dataset do demonstracji:")
print(df_sprzedaz.head())
print(f"\nKształt: {df_sprzedaz.shape}")

In [ ]:
# === DEMONSTRACJA: Prompt → wygenerowany kod (symulacja) ===

dobry_prompt = """
Mam DataFrame pandas `df_sprzedaz` z kolumnami:
- data_zamowienia (datetime)
- klient_id (int)
- produkt_kategoria (str: 'elektronika', 'odzież', 'dom_i_ogrod', 'sport', 'ksiazki')
- wartosc_zamowienia (float, PLN)
- status (str: 'zrealizowane', 'anulowane', 'zwrócone')

Zadanie: Napisz funkcję `raport_kategorii(df)` która:
1. Filtruje tylko status='zrealizowane'
2. Grupuje per kategoria: suma przychodu, liczba zamówień, średnia wartość
3. Dodaje kolumnę 'udzial_pct' (% przychodu tej kategorii)
4. Sortuje malejąco po przychodzie
5. Zwraca DataFrame

Ograniczenia: tylko pandas i numpy.
"""

# Symulowany wygenerowany kod (dokładnie taki jaki dałoby API):
wygenerowany_kod = '''
def raport_kategorii(df):
    """Raport sprzedaży per kategoria — tylko zamówienia zrealizowane."""
    df_real = df[df['status'] == 'zrealizowane'].copy()

    raport = (
        df_real.groupby('produkt_kategoria')['wartosc_zamowienia']
        .agg(
            przychod='sum',
            zamowienia='count',
            sr_wartosc='mean'
        )
        .reset_index()
    )

    raport['przychod'] = raport['przychod'].round(2)
    raport['sr_wartosc'] = raport['sr_wartosc'].round(2)
    raport['udzial_pct'] = (raport['przychod'] / raport['przychod'].sum() * 100).round(1)

    return raport.sort_values('przychod', ascending=False).reset_index(drop=True)
'''

print("=== PROMPT (co wysyłamy do AI) ===")
print(dobry_prompt)
print("=" * 55)
print("\n=== KOD WYGENEROWANY PRZEZ AI ===")
print(wygenerowany_kod)

In [ ]:
# === URUCHOMIENIE WYGENEROWANEGO KODU ===
# (dokładnie tak samo jak w prawdziwym workflow — wklejamy i uruchamiamy)

def raport_kategorii(df):
    """Raport sprzedaży per kategoria — tylko zamówienia zrealizowane."""
    df_real = df[df['status'] == 'zrealizowane'].copy()

    raport = (
        df_real.groupby('produkt_kategoria')['wartosc_zamowienia']
        .agg(
            przychod='sum',
            zamowienia='count',
            sr_wartosc='mean'
        )
        .reset_index()
    )

    raport['przychod'] = raport['przychod'].round(2)
    raport['sr_wartosc'] = raport['sr_wartosc'].round(2)
    raport['udzial_pct'] = (raport['przychod'] / raport['przychod'].sum() * 100).round(1)

    return raport.sort_values('przychod', ascending=False).reset_index(drop=True)


# Uruchamiamy — weryfikacja czy kod działa
wynik = raport_kategorii(df_sprzedaz)
print("Raport per kategoria (wygenerowany przez AI, uruchomiony i zweryfikowany):")
print(wynik.to_string(index=False))
print(f"\nKontrola: suma udziałów = {wynik['udzial_pct'].sum():.1f}% (powinno być ~100%)")

In [ ]:
# === WIZUALIZACJA WYNIKÓW Z RAPORTU ===

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

kolory = ['#2ecc71', '#3498db', '#e74c3c', '#f39c12', '#9b59b6']

# Panel 1: Przychód per kategoria
axes[0].barh(
    wynik['produkt_kategoria'],
    wynik['przychod'],
    color=kolory[:len(wynik)],
    alpha=0.85,
    edgecolor='white'
)
axes[0].set_xlabel('Przychód (PLN)')
axes[0].set_title('Przychód per kategoria\n(tylko zamówienia zrealizowane)', fontsize=11)
axes[0].invert_yaxis()
for i, (v, u) in enumerate(zip(wynik['przychod'], wynik['udzial_pct'])):
    axes[0].text(v + 1000, i, f'{v:,.0f} PLN ({u}%)', va='center', fontsize=9)
axes[0].set_xlim(0, wynik['przychod'].max() * 1.35)

# Panel 2: Liczba zamówień
axes[1].bar(
    wynik['produkt_kategoria'],
    wynik['zamowienia'],
    color=kolory[:len(wynik)],
    alpha=0.85,
    edgecolor='white'
)
axes[1].set_ylabel('Liczba zamówień')
axes[1].set_title('Liczba zamówień per kategoria', fontsize=11)
axes[1].tick_params(axis='x', rotation=15)

plt.suptitle('Analiza sprzedaży — raport kategorii\n(kod wygenerowany przez AI, zweryfikowany)',
             fontsize=12, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()
plt.close()

---
## Sekcja 6: Interpretacja wyników statystycznych przez AI

Technika 2: Wyniki analizy + prompt → tekst dla decydenta

In [ ]:
# === SEKCJA 6: INTERPRETACJA WYNIKÓW A/B TESTU ===

# Symulowane wyniki A/B testu (jak z W12)
np.random.seed(42)
koszyk_A = np.random.normal(loc=320, scale=75, size=150)
koszyk_B = np.random.normal(loc=358, scale=80, size=150)

# Obliczenia (prawdziwe — bez AI)
srednia_A = koszyk_A.mean()
srednia_B = koszyk_B.mean()
roznica = srednia_B - srednia_A
pct_wzrost = roznica / srednia_A * 100

# Uproszczony t-test ręcznie (scipy nie jest wymagane w tej sekcji)
n_A, n_B = len(koszyk_A), len(koszyk_B)
se_diff = np.sqrt(koszyk_A.var() / n_A + koszyk_B.var() / n_B)
t_stat = roznica / se_diff
# p-wartość przybliżona (dla dużych prób rozkład t ≈ normalny)
from scipy import stats as scipy_stats
p_val = 2 * (1 - scipy_stats.norm.cdf(abs(t_stat)))

# 95% CI dla różnicy
z_95 = 1.96
ci_low = roznica - z_95 * se_diff
ci_high = roznica + z_95 * se_diff

print("=== Wyniki A/B testu kampanii e-mailowej ===")
print(f"Wersja A (n={n_A}): x̄ = {srednia_A:.2f} PLN")
print(f"Wersja B (n={n_B}): x̄ = {srednia_B:.2f} PLN")
print(f"Różnica:             {roznica:+.2f} PLN ({pct_wzrost:+.1f}%)")
print(f"t-stat:              {t_stat:.4f}")
print(f"p-wartość:           {p_val:.6f}")
print(f"95% CI:              [{ci_low:.2f}, {ci_high:.2f}] PLN")

In [ ]:
# === MOCK AI: INTERPRETACJA WYNIKÓW ===

wyniki_do_interpretacji = f"""
A/B test kampanii e-mailowej:
- Wersja A (n={n_A}): średnia wartość koszyka = {srednia_A:.2f} PLN
- Wersja B (n={n_B}): średnia wartość koszyka = {srednia_B:.2f} PLN
- Welch's t-test: t = {t_stat:.2f}, p = {p_val:.6f}
- 95% CI dla różnicy (B-A): [{ci_low:.1f}, {ci_high:.1f}] PLN
"""

# Mock odpowiedź — dokładnie jaka przyszłaby z prawdziwego API
def mock_interpretuj_ab_test(wyniki: str, klientow_miesiecznie: int = 50000) -> str:
    """Symuluje interpretację AI wyników A/B testu."""
    szac_wzrost_miesiecznie = roznica * klientow_miesiecznie
    szac_wzrost_rocznie = szac_wzrost_miesiecznie * 12

    return f"""**Raport: A/B test kampanii e-mail**

**Główny wniosek:** Personalizowany email (wersja B) statystycznie istotnie
zwiększa wartość koszyka o ok. {roznica:.0f} PLN na transakcję.

**Co to znaczy w praktyce:** Klienci którzy otrzymali spersonalizowaną wiadomość
kupowali średnio o {roznica:.0f} PLN więcej (+{pct_wzrost:.1f}%). Wynik jest
statystycznie istotny (p < 0.001) — szansa przypadku jest mniejsza niż 0.1%.
Nawet w pesymistycznym scenariuszu (dolna granica CI = {ci_low:.0f} PLN)
personalizacja przynosi pozytywny efekt.

**Szacunek finansowy:** Przy {klientow_miesiecznie:,} emailach miesięcznie:
- Dodatkowy przychód miesięcznie: ~{szac_wzrost_miesiecznie:,.0f} PLN
- Dodatkowy przychód rocznie:     ~{szac_wzrost_rocznie:,.0f} PLN

**Rekomendacja:** Wdrożyć wersję B jako standard kampanii e-mailowych.

**Zastrzeżenie:** Wyniki dotyczą jednej kampanii. Zalecamy powtórzenie
testu w innym okresie sezonowym przed pełnym wdrożeniem."""


interpretacja = mock_interpretuj_ab_test(wyniki_do_interpretacji)

print("=== INTERPRETACJA AI (mock — identyczna z prawdziwą odpowiedzią modelu) ===")
print(interpretacja)

In [ ]:
# === WIZUALIZACJA A/B TESTU ===

fig, axes = plt.subplots(1, 3, figsize=(14, 5))

# Panel 1: Histogramy
axes[0].hist(koszyk_A, bins=25, alpha=0.6, color='#3498db', density=True,
             label=f'Wersja A\n(x̄={srednia_A:.0f} PLN)')
axes[0].hist(koszyk_B, bins=25, alpha=0.6, color='#e74c3c', density=True,
             label=f'Wersja B\n(x̄={srednia_B:.0f} PLN)')
axes[0].axvline(srednia_A, color='#3498db', linestyle='--', lw=1.8)
axes[0].axvline(srednia_B, color='#e74c3c', linestyle='--', lw=1.8)
axes[0].set_title(f'Rozkład wartości koszyka\np={p_val:.4f}', fontsize=10)
axes[0].set_xlabel('Wartość koszyka (PLN)')
axes[0].set_ylabel('Gęstość')
axes[0].legend(fontsize=9)

# Panel 2: Boxplot
bp = axes[1].boxplot(
    [koszyk_A, koszyk_B],
    labels=['Wersja A', 'Wersja B'],
    patch_artist=True,
    medianprops=dict(color='black', linewidth=2)
)
bp['boxes'][0].set_facecolor('#3498db')
bp['boxes'][0].set_alpha(0.6)
bp['boxes'][1].set_facecolor('#e74c3c')
bp['boxes'][1].set_alpha(0.6)
axes[1].set_title('Boxplot porównawczy', fontsize=10)
axes[1].set_ylabel('Wartość koszyka (PLN)')
axes[1].grid(True, axis='y', alpha=0.3)

# Panel 3: Przedział ufności
axes[2].errorbar(
    x=['Różnica B–A'],
    y=[roznica],
    yerr=[[roznica - ci_low], [ci_high - roznica]],
    fmt='o', color='#27ae60', markersize=12,
    capsize=15, linewidth=2.5,
    label=f'+{roznica:.1f} PLN'
)
axes[2].axhline(0, color='red', linestyle='--', lw=1.5, label='H₀: brak różnicy')
axes[2].set_title(f'95% CI dla różnicy\n(cały przedział > 0 → wdrożyć B)', fontsize=10)
axes[2].set_ylabel('Różnica (PLN)')
axes[2].legend(fontsize=9)
axes[2].grid(True, axis='y', alpha=0.3)

plt.suptitle('A/B Test kampanii e-mailowej — wyniki + interpretacja AI',
             fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()
plt.close()

---
## Sekcja 7: Czyszczenie danych z pomocą AI

Technika 3: AI generuje słownik mapowania dla niejednorodnych wartości

In [ ]:
# === SEKCJA 7: CZYSZCZENIE OPISÓW ===

# Realistyczny problem: kolumna statusów wpisywana ręcznie przez pracowników
np.random.seed(42)

# 200 rekordów z niejednorodnie wpisanymi statusami
warianty_raw = [
    "zrealizowane", "Zrealizowane", "ZREALIZOWANE", "zrealizowanoe", "zrealiz.",
    "anulowane", "Anulowane", "ANULOWANE", "anulowanie", "anulow.",
    "zwrot", "Zwrot", "ZWROT", "zwrócone", "zwrocone", "zwrót",
    "w realizacji", "W Realizacji", "W trakcie realizacji", "w_realizacji",
    "oczekuje", "oczekuje na płatność", "Oczekuje na płatnosc", "do zapłaty",
]

kolumna_statusow = pd.Series(
    np.random.choice(warianty_raw, size=200)
)

print("Unikalne wartości w surowej kolumnie statusów:")
print(f"Liczba unikalnych: {kolumna_statusow.nunique()}")
print(kolumna_statusow.value_counts().head(10))

In [ ]:
# === MOCK AI: NORMALIZACJA KATEGORII ===

# To jest odpowiedź AI na prompt:
# "Napisz słownik Python mapujący warianty statusów na 5 docelowych kategorii"

MAPA_STATUSOW = {
    # Zrealizowane
    "zrealizowane": "zrealizowane",
    "Zrealizowane": "zrealizowane",
    "ZREALIZOWANE": "zrealizowane",
    "zrealizowanoe": "zrealizowane",    # literówka
    "zrealiz.": "zrealizowane",
    # Anulowane
    "anulowane": "anulowane",
    "Anulowane": "anulowane",
    "ANULOWANE": "anulowane",
    "anulowanie": "anulowane",
    "anulow.": "anulowane",
    # Zwrócone
    "zwrot": "zwrócone",
    "Zwrot": "zwrócone",
    "ZWROT": "zwrócone",
    "zwrócone": "zwrócone",
    "zwrocone": "zwrócone",
    "zwrót": "zwrócone",
    # W realizacji
    "w realizacji": "w_realizacji",
    "W Realizacji": "w_realizacji",
    "W trakcie realizacji": "w_realizacji",
    "w_realizacji": "w_realizacji",
    # Oczekuje na płatność
    "oczekuje": "oczekuje_platnosc",
    "oczekuje na płatność": "oczekuje_platnosc",
    "Oczekuje na płatnosc": "oczekuje_platnosc",
    "do zapłaty": "oczekuje_platnosc",
}

# Zastosowanie
kolumna_czysta = kolumna_statusow.map(MAPA_STATUSOW).fillna("nieznany")

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Przed czyszczeniem
top_przed = kolumna_statusow.value_counts().head(8)
axes[0].barh(top_przed.index[::-1], top_przed.values[::-1],
             color='#e74c3c', alpha=0.75)
axes[0].set_title(f'Przed czyszczeniem\n({kolumna_statusow.nunique()} unikalnych wartości)', fontsize=10)
axes[0].set_xlabel('Liczba rekordów')

# Po czyszczeniu
po_czyszczeniu = kolumna_czysta.value_counts()
kolory_po = ['#2ecc71', '#3498db', '#e74c3c', '#f39c12', '#9b59b6', '#95a5a6']
axes[1].bar(po_czyszczeniu.index, po_czyszczeniu.values,
            color=kolory_po[:len(po_czyszczeniu)], alpha=0.85, edgecolor='white')
axes[1].set_title(f'Po czyszczeniu (słownik AI)\n({kolumna_czysta.nunique()} unikalnych wartości)', fontsize=10)
axes[1].set_ylabel('Liczba rekordów')
axes[1].tick_params(axis='x', rotation=20)

plt.suptitle('Czyszczenie kategorii z pomocą AI — mapowanie wariantów', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()
plt.close()

print(f"\nPrzed: {kolumna_statusow.nunique()} unikalnych wartości → Po: {kolumna_czysta.nunique()} unikalnych wartości")
print(f"Pokrycie mapy: {(~kolumna_czysta.eq('nieznany')).mean()*100:.1f}% rekordów zmapowanych poprawnie")

---
## Sekcja 8: Automatyczne executive summary

Technika 4: Dane z analizy → prompt z szablonem → gotowy raport

In [ ]:
# === SEKCJA 8: AUTOMATYCZNE EXECUTIVE SUMMARY ===

# Zbieramy dane z poprzednich sekcji do słownika
raport_kat = raport_kategorii(df_sprzedaz)

przychod_total = raport_kat['przychod'].sum()
top_kategoria = raport_kat.iloc[0]['produkt_kategoria']
top_udzial = raport_kat.iloc[0]['udzial_pct']

# Status breakdown
status_counts = df_sprzedaz['status'].value_counts(normalize=True) * 100
wskaznik_zwrotow = status_counts.get('zwrócone', 0)
wskaznik_anulowanych = status_counts.get('anulowane', 0)

dane_do_raportu = {
    'przychod_total': przychod_total,
    'top_kategoria': top_kategoria,
    'top_udzial': top_udzial,
    'wskaznik_zwrotow': wskaznik_zwrotow,
    'wskaznik_anulowanych': wskaznik_anulowanych,
    'ab_test_roznica': roznica,
    'ab_test_p': p_val,
}

print("Dane do raportu:")
for k, v in dane_do_raportu.items():
    if isinstance(v, float):
        print(f"  {k}: {v:.2f}")
    else:
        print(f"  {k}: {v}")

In [ ]:
# === MOCK AI: EXECUTIVE SUMMARY ===

def mock_executive_summary(dane: dict) -> str:
    """Symuluje wygenerowanie executive summary przez model AI."""
    p = dane['przychod_total']
    top = dane['top_kategoria']
    top_u = dane['top_udzial']
    zwr = dane['wskaznik_zwrotow']
    anu = dane['wskaznik_anulowanych']
    ab_r = dane['ab_test_roznica']
    ab_p = dane['ab_test_p']

    ab_wniosek = (
        f"potwierdzono statystycznie (p={ab_p:.4f}) — wzrost o +{ab_r:.0f} PLN/transakcję"
        if ab_p < 0.05
        else f"brak statystycznego potwierdzenia (p={ab_p:.4f})"
    )

    zwrot_ocena = (
        "wymaga uwagi (powyżej normy 10%)" if zwr > 10
        else "w normie (poniżej 10%)"
    )

    return f"""**Executive Summary — Analiza zamówień e-commerce**
*(wygenerowane automatycznie przez AI na podstawie analizy danych)*

---

**Wyniki ogólne:**
Łączny przychód z zamówień zrealizowanych wyniósł {p:,.0f} PLN.
Dominującą kategorią jest {top} z udziałem {top_u}% w przychodach.

**Wskaźniki operacyjne:**
- Wskaźnik zwrotów: {zwr:.1f}% — {zwrot_ocena}
- Wskaźnik anulowanych: {anu:.1f}%
- Kampania e-mail (A/B test): efekt personalizacji {ab_wniosek}

**Trzy rekomendacje:**
1. Zbadać przyczyny zwrotów w kategorii {top} — wskaźnik {zwr:.1f}% jest znaczący
2. Wdrożyć personalizację e-mail jako standard (potwierdzona efektywność)
3. Zwiększyć udział kategorii z wyższą marżą (sport, dom_i_ogrod) kosztem niżej marżowych

---
*Uwaga: Executive summary wygenerowany przez AI — wymaga weryfikacji merytorycznej przez analityka.*"""


summary = mock_executive_summary(dane_do_raportu)
print(summary)

---
## Sekcja 9: Ograniczenia AI — ważne dla analityka

AI jest potężnym narzędziem, ale ma konkretne ograniczenia których musisz być świadomy/a.

In [ ]:
# === SEKCJA 9: DEMONSTRACJA OGRANICZEŃ ===

print("=" * 60)
print("OGRANICZENIA LLM — KONKRETNE PRZYKŁADY")
print("=" * 60)

print("""
1. HALUCYNACJE
   Pytanie: 'Jaka funkcja pandas zwraca korelację?'

   Odpowiedź poprawna:  df.corr() lub df['a'].corr(df['b'])
   Możliwa halucynacja: df.correlation() <-- NIE ISTNIEJE
                        df.pearson_corr() <-- NIE ISTNIEJE

   Reguła: ZAWSZE uruchamiaj wygenerowany kod.
           Błąd: AttributeError = halucynacja funkcji

2. KNOWLEDGE CUTOFF
   Pytanie: 'Jakie są najlepsze biblioteki do AI agentów w 2025?'

   Model z cutoff 2024: nie zna nowości z 2025
   Może polecać przestarzałe biblioteki lub wersje API

   Reguła: dla aktualnych bibliotek → sprawdź oficjalną dokumentację

3. MATEMATYKA
   LLM = model językowy, NIE kalkulator
   Może popełniać błędy arytmetyczne

   Przykład: GPT-4 czasem myli 12% z 0.12 w obliczeniach wielokrokowych

   Reguła: obliczenia zawsze wykonaj Pythonem, nie ufaj liczbie z AI

4. PRYWATNOŚĆ DANYCH
   Dane wysłane do publicznego API:
   - Mogą być logowane przez dostawcę
   - Mogą być użyte do trenowania (zależy od ustawień)
   - Mogą być przechowywane poza UE (RODO!)

   Reguła: dane osobowe klientów → lokalne modele lub zgoda prawników
""")

In [ ]:
# === PODSUMOWANIE SEKCJI ===

print("=" * 60)
print("PODSUMOWANIE NOTEBOOK'U W14")
print("=" * 60)
print("""
OMÓWIONE TECHNIKI:

1. Tokenizacja — LLM widzi tokeny, nie znaki
   Języki fleksyjne (PL) = więcej tokenów = wyższy koszt

2. Temperatura
   KOD → temperatura 0.0-0.2
   RAPORTY → temperatura 0.3-0.7
   BRAINSTORMING → temperatura 1.0-1.5

3. Modele: GPT (OpenAI), Claude (Anthropic), Gemini (Google)
   Różne mocne strony — podobna struktura API

4. Generowanie kodu: KONTEKST + ZADANIE + FORMAT + OGRANICZENIA
   → Zawsze uruchamiaj i weryfikuj wygenerowany kod

5. Interpretacja wyników: wyniki numeryczne + prompt → tekst dla zarządu

6. Czyszczenie danych: AI generuje słownik mapowania w sekundy
   → Ręcznie sprawdzasz mapowanie, nie tworzysz go od zera

7. Executive summary: template + dane = raport

NARZĘDZIA DLA STUDENTA (DARMOWE):
   ✓ Claude.ai — free tier (~20 wiad/kilka h)
   ✓ ChatGPT — free tier (GPT-4o mini)
   ✓ GitHub Copilot — darmowy dla studentów (GitHub Education)
""")